In [ ]:
import sys
sys.path.append('..')

import json
import psycopg2
import matplotlib.pyplot as plt
import numpy as np
import datetime
import pandas as pd
import foodcosts.core as core

In [ ]:
core.get_sales_by_day

In [ ]:
today = datetime.datetime.now().date()
start_date = (today - datetime.timedelta(days=1000)).strftime("%Y-%m-%d")
end_date = today.strftime("%Y-%m-%d")

FRUIT_BASKET_PRODUCT_ID = 53445
    
print(core.get_sales_by_day(start_date, end_date, [], FRUIT_BASKET_PRODUCT_ID))

[]


In [ ]:
def get_last_week_best_selling_products(shops=[]):
    today = datetime.datetime.now().date()
    start_date = (today - datetime.timedelta(days=7)).strftime("%Y-%m-%d")
    end_date = today.strftime("%Y-%m-%d")    
    return get_best_selling_products(start_date, end_date, shops)

In [ ]:
def display_top_selling_products_per_column(shops, top_n=10):
    data = {'rank': range(1, top_n+1)}
    for shop in shops:
        best_selling_products = get_last_week_best_selling_products(shops=[shop])
        top_selling_products = best_selling_products[:top_n]
        data[shop] = [product for [shop, product, amount] in top_selling_products]                
    
    def get_color_for_product(existing_bag, product_id):
        def generate_color(product_name):
            id = hash(product_name)
            red = (id * 13) % 256
            green = (id * 7) % 256
            blue = (id * 19) % 256
            return f"#{red:02x}{green:02x}{blue:02x}99"

        if product_id not in existing_bag:
            existing_bag[product_id] = generate_color(product_id)
        return existing_bag[product_id]
        
    existing_bag = {}
    df = pd.DataFrame(data)
    df_style = df.style.applymap(lambda x: 'background-color: '+ get_color_for_product(existing_bag, x))
    display(df_style)


In [ ]:
def get_best_selling_products(start_date, end_date, shops=[]):
    translation_dict = core.get_product_names()
    query = """
    SELECT
       v_shop.name AS shop_name, 
       v_product.name AS product_name, 
       SUM(v_product.price * v_pos_order_line.quantity) AS total_sales,
       v_product.id AS product_id
    FROM v_pos_order_line
    LEFT JOIN v_product ON v_product.id = v_pos_order_line.product_id
    LEFT JOIN v_pos_order ON v_pos_order.id = v_pos_order_line.order_id
    LEFT JOIN v_pos_session ON v_pos_session.id = v_pos_order.session_id
    LEFT JOIN v_pos ON v_pos.id = v_pos_session.pos_id
    LEFT JOIN v_shop ON v_shop.id = v_pos.shop_id
    WHERE v_pos_order.created_at BETWEEN '{}' AND '{}'
    """.format(start_date, end_date)
    
    if len(shops) > 0:
        query += "AND v_shop.name in ('{}')".format("','".join(shops))
    
    query += "GROUP BY v_shop.id, v_shop.name, v_product.id, v_product.name ORDER BY total_sales DESC" 
    return [[row[0], translation_dict.get(row[3],'None')+ f"({str(row[3])})", row[2]] for row in core.make_sql_query(query)]

In [ ]:
get_best_selling_products(start_date = '2022-01-01', end_date = '2022-01-31', shops = ['Woluwe'])

[['Woluwe', 'PAVLOVA AUX FRUITS ROUGES JETABLE(36501)', Decimal('5555.60000')],
 ['Woluwe', 'BAR EPICE(50689)', Decimal('2605.98400')],
 ['Woluwe', 'POULET AU ROMARIN (18191)', Decimal('2463.51300')],
 ['Woluwe', 'SAUMON MI CUIT EPICE(53011)', Decimal('2291.96000')],
 ['Woluwe', 'CHAMPIGNONS DES BOIS SAUTES(51532)', Decimal('2194.29370')],
 ['Woluwe',
  'SALADE GO GREEN SUCRINE / HARICOTS VERTS / COURGETTES / EDAMAME / AVOCATS(52986)',
  Decimal('2154.18580')],
 ['Woluwe', 'DIVERS PIECE BALANCE(54437)', Decimal('2000.00000')],
 ['Woluwe',
  'SCAMPIS / OIGNONS / TOMATES SECHEES(50615)',
  Decimal('1930.3680')],
 ['Woluwe', 'THON ALBACORE EPICE CUIT(18440)', Decimal('1727.9400')],
 ['Woluwe',
  "FILET DE BOEUF / CROUTE D'HERBES / GRAINES DE MOUTARDE(40750)",
  Decimal('1713.5040000')],
 ['Woluwe', 'COQUELET CUIT + SAUCE COMPTOIR(55577)', Decimal('1592.10000')],
 ['Woluwe', 'POMMES DE TERRE AU PARMESAN(50645)', Decimal('1573.84500')],
 ['Woluwe', 'ASPERGE VERTE ET SUGAR SNAPS (40762)', De

In [ ]:
def get_last_week_best_selling_products(shops=[]):
    today = datetime.datetime.now().date()
    start_date = (today - datetime.timedelta(days=7)).strftime("%Y-%m-%d")
    end_date = today.strftime("%Y-%m-%d")    
    return get_best_selling_products(start_date, end_date, shops)

In [ ]:
display_top_selling_products_per_column(shops=['Fort Jaco','Wemmel','La Hulpe','CHATELAIN','Woluwe', 'LATEM'], top_n=20)

,rank,Fort Jaco,Wemmel,La Hulpe,CHATELAIN,Woluwe,LATEM
0,1,PAVLOVA AUX FRUITS ROUGES JETABLE(36501),"LEGUMES / SALADES 21,7 EURO/KG(54420)","LEGUMES / SALADES 21,7 EURO/KG(54420)",DIVERS PIECE BALANCE(54437),PAVLOVA AUX FRUITS ROUGES JETABLE(36501),"LEGUMES / SALADES 21,7 EURO/KG(54420)"
1,2,GAMBAS GEANTES (50610),GAMBAS GEANTES (50610),LOTTE EPICEE(51181),DIVERS POIDS BALANCE(53935),GIFT CARD FTK(54569),JARDINIERE LEGUMES MARCHE / AIL / HERBES / CITRON(18521)
2,3,TIRADITO DE SAUMON / SESAME / PIMENTS(53784),BAR EPICE(50689),PAVLOVA AUX FRUITS ROUGES JETABLE(36501),PLAT DU JOUR 10.9(19322),POULET AU ROMARIN (18191),DOS DE CABILLAUD EPICE(53764)
3,4,POULET AU ROMARIN (18191),PAVLOVA AUX FRUITS ROUGES JETABLE(36501),BAR EPICE(50689),"LEGUMES / SALADES 24,7 EURO/KG(54419)",BAR EPICE(50689),PAVLOVA AUX FRUITS ROUGES EN VASE(19842)
4,5,"LEGUMES / SALADES 21,7 EURO/KG(54420)",TARTIFLETTE(57765),ASPERGE VERTE ET SUGAR SNAPS (40762),CHAMPIGNONS DES BOIS SAUTES(51532),"LEGUMES / SALADES 21,7 EURO/KG(54420)",BAR EPICE(50689)
5,6,SCAMPIS / OIGNONS / TOMATES SECHEES(50615),FILET DE CABILLAUD EN CROUTE DE PERSIL ET MOUTARDE(53780),CHAMPIGNONS DES BOIS SAUTES(51532),TOMATE CREVETTES GRISES(18655),POMMES DE TERRE AU PARMESAN(50645),SAUMON MI CUIT EPICE(53011)
6,7,DOS DE CABILLAUD EPICE(53764),BUFFET TOUR DU MONDE (prix/pers)(19430),MAGRET DE CANARD EN SAUCE(40752),SCAMPIS / OIGNONS / TOMATES SECHEES(50615),BUFFET DOLCE FARNIENTE (prix/pers)(19531),OSSO BUCO DE VEAU CLASSICO(51282)
7,8,COQUELET CUIT + SAUCE COMPTOIR(55577),TARTARE DE SAUMON / SESAME NOIRE / WAKAME 400G (Gluten Free)(55885),POMMES DE TERRE AU PARMESAN(50645),PAVLOVA AUX FRUITS ROUGES JETABLE(36501),COCKTAIL LE 8 PCS (19429),GAMBAS GEANTES (50610)
8,9,BELLES DE FONTENAY / GROS SEL / ROMARIN(18602),MAGRET DE CANARD EN SAUCE(40752),SAUCE COMPTOIR CHAUDE(38021),PAVLOVA AUX FRUITS ROUGES EN VASE(19842),SAUMON MI CUIT EPICE(53011),TIRADITO DE SAUMON / SESAME / PIMENTS(53784)
9,10,PAIN DE VIANDE NATURE(51722),SAUMON MI-CUIT LAQUE AU SOJA / SESAME(40729),TARTARE DE SAUMON / SESAME NOIRE / WAKAME - 200G(43712),GAMBAS GEANTES (50610),ASPERGE VERTE ET SUGAR SNAPS (40762),WATERZOOI POISSON(51330)
